In [21]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [23]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

# Divida os dados de treinamento em treinamento e validação
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
    
train_encodings = tokenizer(train_data['premise'], train_data['hypothesis'], padding=True, truncation=True, return_tensors='pt').to(torch_device)
val_encodings = tokenizer(val_data['premise'], val_data['hypothesis'], padding=True, truncation=True, return_tensors='pt').to(torch_device)
test_encodings = tokenizer(test_data['premise'], test_data['hypothesis'], padding=True, truncation=True, return_tensors='pt').to(torch_device)

# Converta as labels para tensores PyTorch
train_labels = torch.tensor(train_data['label'].values)
val_labels = torch.tensor(val_data['label'].values)

# Crie conjuntos de dados PyTorch
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'])

ImportError: 
BertForSequenceClassification requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFBertForSequenceClassification".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


In [ ]:
# Carregue os dados em DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# Defina otimizador e função de perda
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# Treinamento do modelo
num_epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
# Avaliação do modelo
model.eval()
val_preds = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc='Validation'):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        val_preds.extend(preds)

val_true_labels = val_labels.cpu().numpy()

In [ ]:
# Avaliação e impressão de métricas
accuracy = accuracy_score(val_true_labels, val_preds)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(val_true_labels, val_preds))

# Predições no conjunto de teste
model.eval()
test_preds = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        test_preds.extend(preds)

# Gere o arquivo de submissão
submission_df = pd.DataFrame({'id': test_data['id'], 'prediction': test_preds})
submission_df.to_csv('submission.csv', index=False)